In [ ]:
%pip install numpy -q
import numpy as np

# Exercício 03


## Erro no algoritmo
O algoritmo está faltando uma identação de referência para o laço de:

> Para j = k + 1, ..., n

, na parte de eliminação, que deveria ser seguido por uma identação com apenas 

> $a_{ij} = a_{ij} - ma_{kj}$.



## Resposta

A eliminação de Gauss transforma a matriz original em uma matriz triangular superior. Para cada pivô na linha $ i $ (diagonal principal), o algoritmo realiza operações nas linhas abaixo para eliminar os elementos abaixo do pivô. Para contar as operações, deve-se considerar:

1. As multiplicações para calcular os multiplicadores que zeram os elementos.
2. As subtrações para atualizar as entradas da matriz após a multiplicação.

Assim, para cada linha $ i $, devemos eliminar os elementos abaixo do pivô na coluna $ i $. 

Desta forma, supondo que se está na coluna $ j $ com o pivô na posição $ A_{jj} $. Então, o número de linhas a serem atualizadas é $ n - j $.

Em cada uma dessas linhas, também se realizam operações nas colunas restantes.

Para cada linha abaixo da linha $j$, realiza-se uma multiplicação e uma subtração para cada coluna à direita da coluna $j$.

O número de operações realizadas na coluna $ j $ é proporcional ao número de linhas e colunas restantes, ou seja, $ (n-j) $ linhas e $ (n-j+1) $ colunas. Assim, o número total de operações para o algoritmo de eliminação de Gauss é dado por:

$C(n) = \sum_{j=1}^{n-1} (n-j)(n-j+1)$

Assim, 

$
C(n) = \sum_{j=1}^{n-1} \left[ (n-j)^2 + (n-j) \right]
$

Em que

$
\sum_{j=1}^{n-1} (n-j)^2 = \sum_{k=1}^{n-1} k^2 = \frac{(n-1)n(2n-1)}{6}
$

e

$
\sum_{j=1}^{n-1} (n-j) = \sum_{k=1}^{n-1} k = \frac{(n-1)n}{2}
$

Ou seja, 

$C(n) = \frac{(n-1)n(2n-1)}{6} + \frac{(n-1)n}{2}$

$C(n) = \frac{(n-1)n(2n-1)}{6} + \frac{3(n-1)n}{6}$

$C(n) = \frac{(n-1)n\left[ (2n-1) + 3 \right]}{6} = \frac{(n-1)n(2n+2)}{6}$

$C(n) = \frac{(n-1)n(2n+2)}{6} = \frac{n(n-1)(2n+2)}{6}$

$C(n) = \frac{2n^3}{3} + \frac{n^2}{2} - \frac{7n}{6}$


# Exercicio 14

## Algoritmo de Eliminação de Gauss

Utilizando dados do exemplo 2.

In [ ]:

X = np.matrix([[3,  2,  4],
               [1,  1,  2],
               [4,  3, -2]
               ])

y = np.matrix([[1], 
               [2],
               [3]
               ])


### Algoritmo

In [ ]:

class GaussElimination:

    def __init__(self):
        pass

    @staticmethod
    def _eliminate(X: np.matrix, y: np.matrix) -> tuple[int, np.matrix, np.matrix]:
        n = X.shape[0]
        a = np.float64(X.copy())
        b = np.float64(y.copy())
        for iVetor in range(0, a.shape[0]):
            for iLinha in range(iVetor + 1, n):
                m = (a[iLinha, iVetor] / a[iVetor, iVetor])
                a[iLinha, iVetor] = 0
                for iProximasLinhas in range(iVetor + 1, n):
                    a[iLinha, iProximasLinhas] -= (m * a[iVetor, iProximasLinhas])
                b[iLinha] = b[iLinha] - (m * b[iVetor])
        return (n, a, b)
    
    @staticmethod
    def _solve_system(n: int, a: np.matrix, b: np.matrix) -> np.matrix:
        c = b.copy()
        for iVetor in range(n - 1, -1, -1):
            s = 0
            for iLinha in range(iVetor + 1, n):
                s += a[iVetor, iLinha] * c[iLinha]
            c[iVetor] = (b[iVetor] - s) / a[iVetor, iVetor]
        return c
    
    @staticmethod
    def solve(X: np.matrix, y: np.matrix) -> np.matrix:
        this = GaussElimination()
        n, a, b = this._eliminate(X, y)
        c = this._solve_system(n, a, b)
        return c



### Tempos de execução

In [ ]:

print('Meu algoritmo:')
print(GaussElimination.solve(X, y))
print('')

print('linalg.solve')
print(np.linalg.solve(X, y))
print('')

### Tempos de execução

In [ ]:
%%timeit
GaussElimination.solve(X, y)


In [ ]:
%%timeit
np.linalg.solve(X, y)

# Exercício 20

## Fatoração LU
Utilizando dados do Exemplo 7.

In [ ]:

A = np.matrix([[3, -4,  1],
               [1,  2,  2],
               [4,  0, -3]
               ])

B = np.matrix([[ 9], 
               [ 3],
               [-2]
               ])


### Algoritmo

In [ ]:

class FatoracaoLU:

    def __init__(self) -> None:
        pass

    @staticmethod
    def _pivot_rows(A: np.matrix) -> tuple[np.ndarray, np.matrix]:
        n = A.shape[0]
        a = np.float64(A.copy())

        # Initialize permutation vector with identity
        p = np.arange(n)  
        
        for k in range(n - 1):

            # Find the pivot
            r = np.argmax(np.abs(a[k:n, k])) + k  
            if np.abs(a[r, k]) == 0:
                return p, A

            # Swap rows in matrix and permutation vector
            if r != k:
                a[[k, r], :] = a[[r, k], :]
                p[[k, r]] = p[[r, k]]

            # Eliminate entries below pivot
            #   Store the multiplier in lower triangular part
            #   Then eliminate
            for i in range(k + 1, n):
                m = a[i, k] / a[k, k]
                a[i, k] = m  
                a[i, k + 1:n] -= m * a[k, k + 1:n]  

        return p, a

    @staticmethod
    def _forward_substitution(p: np.ndarray, a: np.matrix, B: np.ndarray) -> np.ndarray:
        n = a.shape[0]
        y = np.zeros(n)
        b = np.float64(B.copy())

        # Apply permutation to b 
        # and perform forward substitution
        for i in range(n):
            y[i] = b[p[i]].item()
            for j in range(i):
                y[i] -= a[i, j] * y[j]
                
        return y

    @staticmethod
    def _backward_substitution(a: np.matrix, y: np.ndarray) -> np.ndarray:
        n = a.shape[0]
        x = np.zeros(n)

        # Perform backward substitution
        for i in range(n - 1, -1, -1):
            x[i] = y[i]
            for j in range(i + 1, n):
                x[i] -= a[i, j] * x[j]
            x[i] /= a[i, i]

        return x

    @staticmethod
    def solve(A: np.matrix, B: np.ndarray) -> np.ndarray:
        fat = FatoracaoLU()

        p, a = fat._pivot_rows(A)
        y = fat._forward_substitution(p, a, B)
        x = fat._backward_substitution(a, y)

        return x


### Resultados

In [ ]:
FatoracaoLU.solve(A, B)

### Tempos de execução

In [ ]:
%%timeit

FatoracaoLU.solve(A, B)